# Sentiment Analysis



## Dataset'in Import Edilmesi

Bütün çalışma boyunca, Kaggle'ın 1-5 arası puanlama skoru bilgisine sahip Amazon Fine Food Reviews dataseti üzerinde işlemler yaptım.

In [2]:
import pandas as pd
df = pd.read_csv("C:/Users/edouc/Desktop/py/Reviews.csv")
df.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


### Preparing and Cleaning Data

In [3]:
import string
from nltk.stem import WordNetLemmatizer
from wordcloud import STOPWORDS
def clear_text(text):
    lemmatizer = WordNetLemmatizer()
    txt = "".join(u for u in text if u not in string.punctuation)
    
    # Converting to Lowercase
    txt = txt.lower()

    #removing stopwords
    txt = txt.split()
    txt = [word for word in txt if word not in STOPWORDS]
    txt = ' '.join(txt)
    
    # Lemmatization
    txt = txt.split()

    txt = [lemmatizer.lemmatize(word) for word in txt]
    txt = ' '.join(txt)
    
    return txt

df['Text'] = df['Text'].apply(clear_text)

In [3]:
df['Text'].head()

0    bought several vitality canned dog food produc...
1    product arrived labeled jumbo salted peanutsth...
2    confection around century light pillowy citrus...
3    looking secret ingredient robitussin believe f...
4    great taffy great price wide assortment yummy ...
Name: Text, dtype: object

# 1-) Logistic Regression - SGD - Naive Bayes Classifier

Bu çalışmayı hazırlarken öncelikle benzer çalışmalara, kullanılan kütüphanelere ve classifier modellerine göz attım. Nihai hedefimiz, eğittiğimiz modelin puansız bir durumu puanlaması olduğu için öncelikle fine-grained sentiment analysis üzerinde çalışmaya odaklandım.

### Logistic Regression

In [14]:
import numpy as np

# random split train(%80) and test(%20) data
index = df.index
df['random_number'] = np.random.randn(len(index))
train = df[df['random_number'] <= 0.8]
test = df[df['random_number'] > 0.8]

<ipython-input-14-bb04d8db3c56>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['random_number'] = np.random.randn(len(index))


In [5]:
# count vectorizer:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(token_pattern=r'\b\w+\b')
train_matrix = vectorizer.fit_transform(train['Text'])
test_matrix = vectorizer.transform(test['Text'])

In [9]:
# Logistic Regression
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()

In [10]:
X_train = train_matrix
X_test = test_matrix
y_train = train['Score']
y_test = test['Score']

In [11]:
lr.fit(X_train,y_train)

D:\Anaconda\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [12]:
predictions = lr.predict(X_test)

In [13]:
# find accuracy, precision, recall:
from sklearn.metrics import confusion_matrix,classification_report
new = np.asarray(y_test)
confusion_matrix(predictions,y_test)

array([[ 7558,  1573,   907,   407,   830],
       [  682,  1757,   648,   249,   315],
       [  479,   898,  3025,  1030,   734],
       [  218,   365,  1152,  4885,  2312],
       [ 2111,  1782,  3245, 10643, 72574]], dtype=int64)

In [14]:
print(classification_report(predictions,y_test))

              precision    recall  f1-score   support

           1       0.68      0.67      0.68     11275
           2       0.28      0.48      0.35      3651
           3       0.34      0.49      0.40      6166
           4       0.28      0.55      0.37      8932
           5       0.95      0.80      0.87     90355

    accuracy                           0.75    120379
   macro avg       0.51      0.60      0.53    120379
weighted avg       0.82      0.75      0.77    120379



### SGD Classifier

In [19]:
from sklearn.linear_model import SGDClassifier
#training classification model with linear classifier with SGD training.
sgd = SGDClassifier()
sgd.fit(X_train, y_train) 

SGDClassifier()

In [20]:
predictions = sgd.predict(X_test)
new = np.asarray(y_test)
confusion_matrix(predictions,y_test)
print(classification_report(predictions,y_test))

              precision    recall  f1-score   support

           1       0.66      0.66      0.66     10921
           2       0.16      0.58      0.25      1744
           3       0.25      0.56      0.34      3982
           4       0.18      0.56      0.27      5401
           5       0.97      0.76      0.85     98331

    accuracy                           0.73    120379
   macro avg       0.44      0.63      0.48    120379
weighted avg       0.87      0.73      0.79    120379



### Naive-Bayes Classifier

In [22]:
from sklearn.naive_bayes import MultinomialNB

nb = MultinomialNB()
nb.fit(X_train, y_train)

MultinomialNB()

In [23]:
predictions = nb.predict(X_test)
new = np.asarray(y_test)
confusion_matrix(predictions,y_test)
print(classification_report(predictions,y_test))

              precision    recall  f1-score   support

           1       0.63      0.60      0.62     11550
           2       0.14      0.58      0.23      1559
           3       0.24      0.48      0.32      4438
           4       0.35      0.44      0.39     13908
           5       0.92      0.79      0.85     88924

    accuracy                           0.72    120379
   macro avg       0.46      0.58      0.48    120379
weighted avg       0.79      0.72      0.75    120379



Bu üç yöntemle oluşturduğumuz modellerde accuracy değerlerinin:

**- Logistic Regression:**  0.75

**- SGD Classifier:** 0.73

**- Naive-Bayes:** 0.72

şeklinde olduğunu görüyoruz. Bu değerler ne kadar düşük değerler olmasa da zaman zaman yanlış sonuçlar almamıza sebep olabiliyor. Bunun denemesini yapacak olursak, yine amazon üzerinden  alınmış 2 puanlık bir incelemenin prediction sonuçlarının farklılık gösterdiğini ve yanlış sonuç verebildiğini aşağıda görüyoruz.

In [27]:
#Logistic Regression
prediction = "We were pretty disappointed with this shampoo and conditioner. Had high hopes for it, and it did not deliver.The smell was okay.It was harsh and stripping on the hair follicles.Left our hair with that squeaky clean feeling, but not in a good way, more in a removed all the oil from your hair kinda away.Even in using the conditioner, my hair still looked frizzy. And I do not have frizzy hair! Mine is fine, thin, and straight. If anything I usually border on having my hair be too silky and need things to volumize it.Just cannot recommend this product."
prediction = vectorizer.transform([prediction]).toarray()
result_lr = lr.predict(prediction)[0]
result_sgd = sgd.predict(prediction)[0]
result_nb = nb.predict(prediction)[0]

print("""
- Logistic Regression: {}

- SGD Classifier: {}

- Naive-Bayes: {}
""".format(result_lr ,result_sgd,result_nb ))


- Logistic Regression: 2

- SGD Classifier: 3

- Naive-Bayes: 3



# 2-) Sadece Positive/Negative Prediction Üzerinde Çalışma

Model eğitimini ve prediction'ı yalnızca pozitif/negatif olgusuna odaklanacak şekilde düzenlediğimizde, accuracy değerlerinin nasıl değişeceğini görmek için sonrasında çalışmamı bu yönde ilerlettim.

In [4]:
#Nötr ifadeleri temizlemek için 3 puanlık incelemeleri kaldırıp yüksek puanı pozitif, düşük puanı negatif şeklinde değerlendirdim.
df = df[df['Score'] != 3]
df['sentiment'] = df['Score'].apply(lambda rating : +1 if rating > 3 else 0)

In [31]:
dfNew = df[['Text','sentiment']]
dfNew.head()

,Text,sentiment
0,bought several vitality canned dog food produc...,1
1,product arrived labeled jumbo salted peanutsth...,-1
2,confection around century light pillowy citrus...,1
3,looking secret ingredient robitussin believe f...,-1
4,great taffy great price wide assortment yummy ...,1


Bu şekilde aynı üç model üzerinde çalıştığımızda;

In [34]:
# random split train and test data
index = df.index
df['random_number'] = np.random.randn(len(index))
train = df[df['random_number'] <= 0.8]
test = df[df['random_number'] > 0.8]

# count vectorizer:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(token_pattern=r'\b\w+\b')
train_matrix = vectorizer.fit_transform(train['Text'])
test_matrix = vectorizer.transform(test['Text'])

# Logistic Regression
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()

X_train = train_matrix
X_test = test_matrix
y_train = train['sentiment']
y_test = test['sentiment']

lr.fit(X_train,y_train)

predictions = lr.predict(X_test)

# find accuracy, precision, recall:
from sklearn.metrics import confusion_matrix,classification_report
new = np.asarray(y_test)
confusion_matrix(predictions,y_test)
print('Logistic Regression Classification Report;\n')
print(classification_report(predictions,y_test))

from sklearn.linear_model import SGDClassifier
#training classification model with linear classifier with SGD training.
sgd = SGDClassifier()
sgd.fit(X_train, y_train) 

predictions = sgd.predict(X_test)
new = np.asarray(y_test)
confusion_matrix(predictions,y_test)
print('SGD Classifier Classification Report;\n')
print(classification_report(predictions,y_test))

from sklearn.naive_bayes import MultinomialNB

nb = MultinomialNB()
nb.fit(X_train, y_train)

predictions = nb.predict(X_test)
new = np.asarray(y_test)
confusion_matrix(predictions,y_test)
print('Naive-Bayes Classifier Classification Report;\n')
print(classification_report(predictions,y_test))

D:\Anaconda\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Logistic Regression Classification Report;

              precision    recall  f1-score   support

          -1       0.74      0.85      0.79     15048
           1       0.98      0.95      0.96     96293

    accuracy                           0.94    111341
   macro avg       0.86      0.90      0.88    111341
weighted avg       0.94      0.94      0.94    111341

SGD Classifier Classification Report;

              precision    recall  f1-score   support

          -1       0.68      0.87      0.76     13606
           1       0.98      0.94      0.96     97735

    accuracy                           0.93    111341
   macro avg       0.83      0.91      0.86    111341
weighted avg       0.94      0.93      0.94    111341

Naive-Bayes Classifier Classification Report;

              precision    recall  f1-score   support

          -1       0.67      0.77      0.72     15150
           1       0.96      0.94      0.95     96191

    accuracy                           0.92    11134

Accuracy değerlerinin önceki duruma göre çok daha yüksek olduğunu görebiliyoruz.

**- Logistic Regression:** 0.94

**- SGD Classifier:** 0.93

**- Naive-Bayes:** 0.92

Predicton sonuçlarına baktığımızda ise 5'lik sistemde puanı 2, yani negatif olan yorumun üç modelde de sonucunun negatif olduğunu aşağıda görüyoruz.

In [36]:
#Logistic Regression
prediction = "We were pretty disappointed with this shampoo and conditioner. Had high hopes for it, and it did not deliver.The smell was okay.It was harsh and stripping on the hair follicles.Left our hair with that squeaky clean feeling, but not in a good way, more in a removed all the oil from your hair kinda away.Even in using the conditioner, my hair still looked frizzy. And I do not have frizzy hair! Mine is fine, thin, and straight. If anything I usually border on having my hair be too silky and need things to volumize it.Just cannot recommend this product."
prediction = vectorizer.transform([prediction]).toarray()
result_lr = lr.predict(prediction)[0]
result_sgd = sgd.predict(prediction)[0]
result_nb = nb.predict(prediction)[0]

print("""
- Logistic Regression: {}

- SGD Classifier: {}

- Naive-Bayes: {}
""".format(result_lr ,result_sgd,result_nb ))


- Logistic Regression: -1

- SGD Classifier: -1

- Naive-Bayes: -1



# 3-) Kendi Text Classifier Modelimizi Oluşturma

Önceki iki çalışmada tam olarak istediğim verimi alamadığım için manuel olarak bir model oluşturmak için araştırmalar yaptım ve örnekleri inceledim. Ancak bu kısımda fine-grained dataset üzerinde çalıştığımda yine istediğim verimi alamadım. Bu sebeple iki farklı model oluşturmayı denedim.

**MODEL 1**       

Bu modelde 0.97 gibi oldukça yüksek bir accuracy değeri elde ettim

In [5]:
new_df = df[['Text','Score','sentiment']]
print(new_df.shape)
new_df.head(5)

(525814, 3)


,Text,Score,sentiment
0,bought several vitality canned dog food produc...,5,1
1,product arrived labeled jumbo salted peanutsth...,1,0
2,confection around century light pillowy citrus...,4,1
3,looking secret ingredient robitussin believe f...,2,0
4,great taffy great price wide assortment yummy ...,5,1


In [6]:
target = new_df['sentiment'].values.tolist()
data = new_df['Text'].values.tolist()

In [7]:
cutoff = int(len(data) * 0.80)
x_train, x_test = data[:cutoff], data[cutoff:]
y_train, y_test = target[:cutoff], target[cutoff:]

In [9]:
from tensorflow.keras.preprocessing.text import Tokenizer

num_words = 10000
tokenizer = Tokenizer(num_words=num_words)

In [10]:
tokenizer.fit_on_texts(data)

In [11]:
x_train_tokens = tokenizer.texts_to_sequences(x_train)
x_test_tokens = tokenizer.texts_to_sequences(x_test)


In [13]:
import numpy as np

num_tokens = [len(tokens) for tokens in x_train_tokens + x_test_tokens]
num_tokens = np.array(num_tokens)

In [14]:
max_tokens = np.mean(num_tokens) + 2 * np.std(num_tokens)
max_tokens = int(max_tokens)
max_tokens

115

In [22]:
np.sum(num_tokens < max_tokens) / len(num_tokens)

0.9603985439718228

In [16]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

x_train_pad = pad_sequences(x_train_tokens, maxlen=max_tokens)
x_test_pad = pad_sequences(x_test_tokens, maxlen=max_tokens)

In [17]:
idx = tokenizer.word_index
inverse_map = dict(zip(idx.values(), idx.keys()))

In [18]:
def tokens_to_string(tokens):
    words = [inverse_map[token] for token in tokens if token!=0]
    text = ' '.join(words)
    return text

In [24]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Dense, GRU, Embedding


model = Sequential()
embedding_size = 50
model.add(Embedding(input_dim=num_words,
                    output_dim=embedding_size,
                    input_length=max_tokens,
                    name='embedding_layer'))

model.add(GRU(units=16, return_sequences=True))
model.add(GRU(units=8, return_sequences=True))
model.add(GRU(units=4))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [25]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_layer (Embedding)  (None, 115, 50)           500000    
_________________________________________________________________
gru (GRU)                    (None, 115, 16)           3264      
_________________________________________________________________
gru_1 (GRU)                  (None, 115, 8)            624       
_________________________________________________________________
gru_2 (GRU)                  (None, 4)                 168       
_________________________________________________________________
dense (Dense)                (None, 1)                 5         
Total params: 504,061
Trainable params: 504,061
Non-trainable params: 0
_________________________________________________________________


In [26]:
y_train = np.array(y_train)
model.fit(x_train_pad, y_train, epochs=5, batch_size=256)

Epoch 1/5
1644/1644 [==============================] - 478s 285ms/step - loss: 0.2318 - accuracy: 0.9177
Epoch 2/5
1644/1644 [==============================] - 467s 284ms/step - loss: 0.1496 - accuracy: 0.9453
Epoch 3/5
1644/1644 [==============================] - 453s 275ms/step - loss: 0.1254 - accuracy: 0.9556
Epoch 4/5
1644/1644 [==============================] - 450s 274ms/step - loss: 0.1069 - accuracy: 0.9631
Epoch 5/5
1644/1644 [==============================] - 248s 151ms/step - loss: 0.0910 - accuracy: 0.9697


In [40]:
#Amazon üzerindeki 4 puanlık bir yorumun oluşturduğumuz model ile elde edilen prediction değeri
prediction = ["I love this product! I expected it to be much bigger though. For my third grade students it is a little too small, but we make it work. If it was larger it would be much better."]
pred_token = tokenizer.texts_to_sequences(prediction)
padded_token = pad_sequences(pred_token, maxlen=max_tokens)
padded_token.shape
model.predict(padded_token)

array([[0.9790913]], dtype=float32)

**MODEL 2**

In [41]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dense, Dropout, SpatialDropout1D
from tensorflow.keras.layers import Embedding

new_df = df[['Text','Score','sentiment']]
print(new_df.shape)
new_df.head(5)

(525814, 3)


,Text,Score,sentiment
0,bought several vitality canned dog food produc...,5,1
1,product arrived labeled jumbo salted peanutsth...,1,0
2,confection around century light pillowy citrus...,4,1
3,looking secret ingredient robitussin believe f...,2,0
4,great taffy great price wide assortment yummy ...,5,1


In [43]:
sentiment_label = df.sentiment.factorize()
sentiment_label

(array([0, 1, 0, ..., 0, 0, 0], dtype=int64),
 Int64Index([1, 0], dtype='int64'))

In [45]:
values = df.Text.values
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(values)
vocab_size = len(tokenizer.word_index) + 1
encoded_docs = tokenizer.texts_to_sequences(values)
padded_sequence = pad_sequences(encoded_docs, maxlen=200)

In [61]:
embedding_vector_length = 16
model = Sequential() 
model.add(Embedding(vocab_size, embedding_vector_length, input_length=200) )
model.add(SpatialDropout1D(0.25))
model.add(LSTM(100, dropout=0.5, recurrent_dropout=0.5))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid')) 
model.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy'])  
print(model.summary()) 

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_9 (Embedding)      (None, 200, 16)           3501872   
_________________________________________________________________
spatial_dropout1d_9 (Spatial (None, 200, 16)           0         
_________________________________________________________________
lstm_9 (LSTM)                (None, 100)               46800     
_________________________________________________________________
dropout_9 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 1)                 101       
Total params: 3,548,773
Trainable params: 3,548,773
Non-trainable params: 0
_________________________________________________________________
None


In [64]:
model.fit(padded_sequence,sentiment_label[0],validation_split=0.2, epochs=5, batch_size=256);

Epoch 1/5
1644/1644 [==============================] - 1550s 943ms/step - loss: 0.1849 - accuracy: 0.9277 - val_loss: 0.1733 - val_accuracy: 0.9320
Epoch 2/5
1644/1644 [==============================] - 1723s 1s/step - loss: 0.1812 - accuracy: 0.9293 - val_loss: 0.1710 - val_accuracy: 0.9327
Epoch 3/5
1644/1644 [==============================] - 1591s 968ms/step - loss: 0.1750 - accuracy: 0.9319 - val_loss: 0.1691 - val_accuracy: 0.9340
Epoch 4/5
1644/1644 [==============================] - 1387s 844ms/step - loss: 0.1710 - accuracy: 0.9331 - val_loss: 0.1697 - val_accuracy: 0.9347
Epoch 5/5
1644/1644 [==============================] - 1383s 841ms/step - loss: 0.1675 - accuracy: 0.9352 - val_loss: 0.1643 - val_accuracy: 0.9363


In [76]:
#Amazon üzerindeki 2 puanlık bir yorumun oluşturduğumuz model ile elde edilen prediction değeri
text = ["We were pretty disappointed with this shampoo and conditioner. Had high hopes for it, and it did not deliver.The smell was okay.It was harsh and stripping on the hair follicles.Left our hair with that squeaky clean feeling, but not in a good way, more in a removed all the oil from your hair kinda away.Even in using the conditioner, my hair still looked frizzy. And I do not have frizzy hair! Mine is fine, thin, and straight. If anything I usually border on having my hair be too silky and need things to volumize it.Just cannot recommend this product."]
tokens = tokenizer.texts_to_sequences([text])
padded_tokens = pad_sequences(tokens,maxlen=max_tokens)
model.predict(padded_tokens)


array([[0.34768367]], dtype=float32)